# Задание №1

В качестве метрик для вывода на дашборд СЕО я выберу следующие варианты:

1) Отношение количества заказов за предыдущий день к этому же дню на предыдущей неделе. Эта метрика позволит заметить проблемы в работе сервиса на следующий день после их появления. Смотреть метрику по текущему дню нет смысла.

2) Отношение положительных отзывов пользователей к отрицательным за прошедшие семь дней. Эта метрика покажет реакцию пользователей на изменения в работе сервиса.

3) Отношение количества покупок по предыдущему месяцу к покупкам в текущем месяце с учётом количества прошедших дней. Эта метрика позволит оценить прирост клиентов по сравнению с предыдущем месяцем. 

4) Конверсия по типам рекламы за последнюю неделю. Эта метрика позволит оценить эффективность работы отдела маркетинга.

5) Выручка за последнюю неделю. Сильное падение значения метрики относительно предыдущего дня может означать наличие проблем в системе функционирования бизнеса. 

# Задание №2

## План

Поиск точек роста будем вести следующим образом:

1) Выгрузка данных

2) Проверка данных на наличие пропусков/нежелательных значений и принятие решений о их исправлении

3) Формирование гипотез

4) Проверка каждой гипотезы

5) Общие выводы

In [22]:
import psycopg2 # импортируем необходимые библиотеки

import pandas as pd

In [23]:
# получаем базу данных
conn = psycopg2.connect(user='analytics', password='HRanalytics',
                        database='hr-analytics', host='rc1c-fhrb9f1e0l9g611h.mdb.yandexcloud.net', port="6432")


In [24]:
# формируем таблицы
orders = pd.read_sql("SELECT  id,created_at,delivery_window_id,item_total,promo_total,cost,total_cost,ship_address_id,shipped_at,state,store_id,total_quantity,total_weight,user_id FROM orders", conn)

delivery_windows = pd.read_sql("SELECT  id,starts_at,ends_at,store_id FROM delivery_windows", conn)

stores = pd.read_sql("SELECT  id,city,retailer_id FROM stores", conn)

addresses = pd.read_sql("SELECT  id,lat,lon FROM addresses", conn)

replacements = pd.read_sql("SELECT  item_id,order_id,state FROM replacements", conn)

cancellations = pd.read_sql("SELECT  item_id,order_id,state FROM cancellations", conn)

In [25]:
tables = [orders, delivery_windows, stores, addresses, replacements, cancellations] # для удобства создаём список

In [26]:
# проверяем таблицы на наличие пропусков
for table in tables:   
    print('next table')
    print()
    table.info()
    print()

next table

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720433 entries, 0 to 720432
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   id                  720433 non-null  int64         
 1   created_at          720433 non-null  datetime64[ns]
 2   delivery_window_id  720433 non-null  int64         
 3   item_total          720433 non-null  int64         
 4   promo_total         720433 non-null  float64       
 5   cost                720433 non-null  float64       
 6   total_cost          720433 non-null  float64       
 7   ship_address_id     720433 non-null  int64         
 8   shipped_at          720433 non-null  datetime64[ns]
 9   state               720433 non-null  object        
 10  store_id            720433 non-null  int64         
 11  total_quantity      720433 non-null  int64         
 12  total_weight        720433 non-null  int64         
 13  user_id          

Судя по всему, данные были заранее подготовлены, поэтому дальнейшая проверка не является необходимой. 

Теперь сформируем первые гипотезы. Начнём с ключевой проблемы - замены и отмены товаров. Проверим в каких магазинах происходят эти проблемы - во всех примерно одинаково или в каком-то большая концентрация. Также проверим как отмены влияют на последующие заказы от данных клиентов.

In [27]:
orders["state"].unique() # смотрим возможные статусы заказов. 


array(['shipped', 'canceled', 'pending', 'shipping', 'ready'],
      dtype=object)

Отдельное внимание стоит уделить отменённым заказам, но пока сосредоточимся на отменённых товарах.

In [28]:
display(orders[orders["id"].isin(cancellations['order_id'])]) # смотрим на заказы с отменёнными товарами

,id,created_at,delivery_window_id,item_total,promo_total,cost,total_cost,ship_address_id,shipped_at,state,store_id,total_quantity,total_weight,user_id
0,756927,2018-08-26 02:06:55,226459,5015,0.0,0.0,0.0,888330,2018-08-26 15:01:42,shipped,1,39,37708,139676
1,756963,2018-08-26 05:43:20,226460,7096,0.0,0.0,0.0,888394,2018-08-26 16:28:41,shipped,1,71,49435,139682
2,756983,2018-08-26 06:12:13,226459,2724,-250.0,299.0,299.0,888420,2018-08-26 15:09:11,shipped,1,32,11590,63967
3,756985,2018-08-26 06:13:59,226462,3471,0.0,0.0,0.0,888422,2018-08-26 18:57:01,shipped,1,13,9915,139687
4,757066,2018-08-26 07:40:58,226458,1254,0.0,299.0,299.0,870688,2018-08-26 13:46:48,shipped,1,24,15022,119006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720421,5413920,2019-12-18 04:51:21,973248,3400,0.0,238.0,158.0,7021516,2019-12-18 09:20:20,shipped,235,21,6114,1813765
720425,5415671,2019-12-18 06:02:39,973252,1582,-300.0,238.0,158.0,7111531,2019-12-18 13:00:56,shipped,235,9,3418,1090946
720428,5423310,2019-12-18 09:16:28,973254,4777,0.0,258.0,258.0,6346125,2019-12-18 15:45:12,shipped,235,71,43286,1676786
720431,5426016,2019-12-18 10:18:37,973254,2876,0.0,238.0,158.0,5606095,2019-12-18 14:28:56,shipped,235,34,13385,1116106


Больше половины заказов должны были содержать в себе больше товара! Это уже тревожный звоночек о качестве работы сервиса. Отмена товара это потеря прибыли для магазина, а также отмена товара огорчает клиентов. Проверим теперь количество замен.

In [29]:
display(orders[orders["id"].isin(replacements['order_id'])])

,id,created_at,delivery_window_id,item_total,promo_total,cost,total_cost,ship_address_id,shipped_at,state,store_id,total_quantity,total_weight,user_id
3,756985,2018-08-26 06:13:59,226462,3471,0.0,0.0,0.0,888422,2018-08-26 18:57:01,shipped,1,13,9915,139687
4,757066,2018-08-26 07:40:58,226458,1254,0.0,299.0,299.0,870688,2018-08-26 13:46:48,shipped,1,24,15022,119006
6,757117,2018-08-26 08:14:10,227237,2985,-350.0,249.0,249.0,778649,2018-08-27 13:56:22,shipped,1,24,21930,125115
10,757167,2018-08-26 08:32:50,227241,6251,0.0,0.0,0.0,888702,2018-08-27 08:01:37,shipped,1,51,24765,139731
12,757184,2018-08-26 08:38:23,227243,3358,-350.0,299.0,299.0,888715,2018-08-27 12:52:10,shipped,1,31,13192,128211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720427,5417553,2019-12-18 07:00:59,973251,2237,0.0,238.0,158.0,6869883,2019-12-18 11:50:45,shipped,235,17,7814,1526596
720428,5423310,2019-12-18 09:16:28,973254,4777,0.0,258.0,258.0,6346125,2019-12-18 15:45:12,shipped,235,71,43286,1676786
720429,5423488,2019-12-18 09:20:18,973253,1864,0.0,238.0,98.0,7120759,2019-12-18 12:44:16,shipped,235,13,4245,1832586
720431,5426016,2019-12-18 10:18:37,973254,2876,0.0,238.0,158.0,5606095,2019-12-18 14:28:56,shipped,235,34,13385,1116106


Видим, что замен тоже было много. Снижение этих параметров положительно скажется на рейтинге компании. Теперь посмотрим на магазины, где были отмены.

In [30]:
cancel = orders[orders["id"].isin(cancellations['order_id'])]

amount = (orders['store_id'].unique()) 

print(len(amount)) # смотрим уникальное количество магазинов

cancel_value = cancel.groupby('store_id')['id'].count().sort_values()
print(cancel_value) #как видим, число магазинов, в которых была замена = 70, при этом
# число замен существенно различается. Попробуем посчитать соотношение заказов без замен к заказам с заменой. 

all_value = orders.groupby('store_id')['id'].count().sort_values()
print(all_value) # Общее число заказов по магазинам


all_value  = all_value.sort_index()

cancel_value  = cancel_value.sort_index()

percentage = cancel_value/all_value *100

print(percentage) # выводим процентное соотношение отмены товаров в каждом магазине

print('Максимальный процент отмен',percentage.max())

print('Минимальный процент отмен',percentage.min())

71
store_id
317        2
254      122
259      128
248      144
252      237
       ...  
12     25863
1      27928
8      29396
2      30446
14     34487
Name: id, Length: 70, dtype: int64
store_id
179        1
317        3
254      193
248      221
259      228
       ...  
1      44557
8      45599
12     47574
2      52511
14     57606
Name: id, Length: 71, dtype: int64
store_id
1      62.679265
2      57.980233
3      63.921234
8      64.466326
9      63.593339
         ...    
259    56.140351
260    70.086957
261    68.893528
262    64.627660
317    66.666667
Name: id, Length: 71, dtype: float64
Максимальный процент отмен 83.73349339735896
Минимальный процент отмен 45.61664190193165


Как видим из полученных данных, процент отмен по магазинам довольно сильно варьируется, но даже минимальные 45% - это довольно много. Другими словами, из каждого второго заказа убирается часть товара. Над этим стоит поработать, возможно ввести электронный учёт товара на складе, и при его снижении автоматически оповещать клиентов, а также выводить уведомление на сайт. Также стоит уделить больше внимания магазинам с высоким соотношением отмены товара на заказ, возможно им нужна консультация. В конце концов они портят репутацию сервису.

Теперь попробуем проследить как ведут себя клиенты после неудачного опыта отмены товара - насколько часто они перестают пользоваться услугами сервиса. Для тестового промежутка будем брать временной промежуток без последнего полугода, определим id пользователя. Затем посмотрим, были ли ещё заказы с данных адресов. 

In [31]:
cancel_time = cancel[cancel['shipped_at']<'2019-06-01 08:00:00'] # заказы с отменой товара до 1 июня 2019 года

check_period = orders[orders['shipped_at']>'2019-06-01 08:00:00'] # период, в котором мы смотрим повторные заказы

re_buy = check_period[check_period['user_id'].isin(cancel_time['user_id'])] # вернувшиеся клиенты

display('Уникальных покупателей в cancel_time', len(cancel_time['user_id'].unique()))

display('Уникальных покупателей в re_buy', len(re_buy['user_id'].unique()))




'Уникальных покупателей в cancel_time'

48045

'Уникальных покупателей в re_buy'

20064

По результатам видим, что среди оставшихся клиентов меньше половины от тех, кому отказали. Также рассчитаем этот параметр для всей массы клиентов

In [32]:
all_time = orders[orders['shipped_at']<'2019-06-01 08:00:00'] # заказы товара до 1 июня 2019 года

re_buy = check_period[check_period['user_id'].isin(all_time['user_id'])] # Вернувшиеся клиенты

display('Уникальных покупателей в all_time', len(all_time['user_id'].unique()))

display('Уникальных покупателей в re_buy', len(re_buy['user_id'].unique()))




'Уникальных покупателей в all_time'

76227

'Уникальных покупателей в re_buy'

26082

Удивительно, судя по данным, среди пользователей, которые отменили как минимум 1 товар из заказа, услугами сервиса остаются пользоваться больше людей, чем в общем случае. Это конечно маловероятно, поэтому такой результат мог получиться в силу неполноты данных. 

Вернёмся к отменённым заказам. Проверим сколько их и соберём о них информацию

In [33]:
display(orders[orders['state']=='canceled']['id'].count()) # Всего 15 полностью отменённых заказов - здесь нечего исследовать. 

15

Проверим насколько быстро организуется доставка в разных магазинах. Критическую отметку по времени примем в 7 дней. Конечно, здесь может играть специфика конкретного магазина, но если брать в расчёт что почти все магазины - обычные продуктовые супермаркеты, то большее время доставки может убавлять интерес у пользователей.

In [34]:
display(orders[orders['shipped_at']-orders['created_at']>'168:00:00'])

,id,created_at,delivery_window_id,item_total,promo_total,cost,total_cost,ship_address_id,shipped_at,state,store_id,total_quantity,total_weight,user_id
22,757446,2018-08-26 10:34:04,244043,7506,-1655.1,645.0,645.0,889063,2018-09-16 14:17:11,shipped,1,72,28371,93129
25,757481,2018-08-26 10:51:49,232428,3051,0.0,299.0,299.0,871657,2018-09-02 16:37:06,shipped,1,18,7522,120391
27,757544,2018-08-26 11:19:15,234090,1144,0.0,0.0,0.0,889242,2018-09-04 19:54:16,shipped,1,19,44420,142466
29,757590,2018-08-26 11:38:23,297165,3696,0.0,498.0,0.0,786156,2018-11-09 16:07:27,shipped,1,44,73125,118410
55,758339,2018-08-26 18:44:17,233263,822,0.0,299.0,299.0,887133,2018-09-03 20:06:58,shipped,1,36,12585,45164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720045,4903091,2019-12-05 16:51:24,958234,2517,0.0,238.0,158.0,6457862,2019-12-14 13:04:27,shipped,235,42,7940,1622691
720047,4904753,2019-12-05 17:32:52,954490,4977,0.0,238.0,158.0,6471357,2019-12-13 13:57:02,shipped,235,76,18452,1225364
720069,4922283,2019-12-06 07:27:21,954488,3183,0.0,238.0,0.0,6324902,2019-12-13 10:46:25,shipped,235,23,16540,1589539
720131,4988408,2019-12-07 14:26:58,965722,4579,0.0,238.0,158.0,6499127,2019-12-16 11:17:57,shipped,235,45,16585,1240921


Как видим, есть магазины, доставка из которых занимает много времени. Посмотрим подробнее на данные магазины

In [35]:
display(orders[orders['shipped_at']-orders['created_at']>'168:00:00'].groupby('store_id')['store_id'].count().sort_values(ascending = False))

store_id
14     7481
2      6865
10     6727
8      6511
1      6317
       ... 
261       6
252       6
254       2
262       2
259       1
Name: store_id, Length: 68, dtype: int64

Можно увидеть, что доставка из определённых магазином иногда происходит с огромной задержкой - с этим надо что-то сделать. Ведь клиент, которому неделю везут продукты, наверняка останется недоволен, и будет пользоваться другими сервисами. Конечно, некоторые специально заказывают заранее, но ведь далеко не все - некоторым клиентам доставляют долго из-за технических проблем или человеческого фактора, а это нужно исправлять. Проверим теперь, у каких именно фирм происходит такая задержка.

In [36]:
long_wait = orders[orders['shipped_at']-orders['created_at']>'168:00:00']

sum_of_fails = orders[orders['shipped_at']-orders['created_at']>'168:00:00'].groupby('store_id')['store_id'].count().sort_values(ascending = False)

fail_stores = stores.copy() # создаём копию таблицы для работы с ней

for index in range(162): # в цикле заменяем значения city на сумму кол-во условно просроченных заказов
    if any(sum_of_fails.index.isin(list(fail_stores[fail_stores.index==index]['id'].values))):   
        fail_stores.loc[index,'city'] = int(sum_of_fails[sum_of_fails.index ==int(fail_stores[fail_stores.index==index]['id'].values)])

print('Разбиение "просроченных заказов" по ритейлерам', fail_stores.groupby('retailer_id')['city'].sum())

Разбиение "просроченных заказов" по ритейлерам retailer_id
1     76027
8      4157
15     4080
16      104
59       14
61        1
Name: city, dtype: int64


Как видим, проблемы в подавляющем большинстве у ритейлера с id 1, а также немного проблем у ритейлеров с id 8 и 15. Конечно, было бы правильнее посмотреть на процент "просроченных" заказов, но в данном случае такое количество не допустимо просто само по себе для выборки из 760 тыс., ведь 76 тыс. потенциально недовольных клиентов - это крайне негативно для репутации компании. Возможно, стоит поговорить именно с ними и выяснить причины. Конечно, сначала хорошо бы выявить сколько из этих заказов действительно просрочено. 

Мы уже проанализировали проблемную область с отсутствием товаров на складе и последующей отменой/заменой товара, а также проблемы с большим временем доставки. Теперь обратим внимание на расчёт скидок. Посмотрим, все ли скидки учитываться как надо.

In [37]:
discount = orders[orders["promo_total"]<0]

print(discount.info())
display(discount[discount['cost']-discount['total_cost']<0.1]) # Смотрим заказы, где скидка не применилась

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97838 entries, 2 to 720430
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  97838 non-null  int64         
 1   created_at          97838 non-null  datetime64[ns]
 2   delivery_window_id  97838 non-null  int64         
 3   item_total          97838 non-null  int64         
 4   promo_total         97838 non-null  float64       
 5   cost                97838 non-null  float64       
 6   total_cost          97838 non-null  float64       
 7   ship_address_id     97838 non-null  int64         
 8   shipped_at          97838 non-null  datetime64[ns]
 9   state               97838 non-null  object        
 10  store_id            97838 non-null  int64         
 11  total_quantity      97838 non-null  int64         
 12  total_weight        97838 non-null  int64         
 13  user_id             97838 non-null  int64    

,id,created_at,delivery_window_id,item_total,promo_total,cost,total_cost,ship_address_id,shipped_at,state,store_id,total_quantity,total_weight,user_id
2,756983,2018-08-26 06:12:13,226459,2724,-250.0,299.0,299.0,888420,2018-08-26 15:09:11,shipped,1,32,11590,63967
6,757117,2018-08-26 08:14:10,227237,2985,-350.0,249.0,249.0,778649,2018-08-27 13:56:22,shipped,1,24,21930,125115
8,757134,2018-08-26 08:21:14,226464,597,-450.0,299.0,299.0,888655,2018-08-26 20:18:37,shipped,1,17,15165,99699
12,757184,2018-08-26 08:38:23,227243,3358,-350.0,299.0,299.0,888715,2018-08-27 12:52:10,shipped,1,31,13192,128211
14,757211,2018-08-26 08:47:56,231596,1761,-450.0,299.0,299.0,838277,2018-09-01 08:19:16,shipped,1,50,45130,114596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719952,5358474,2019-12-16 14:09:33,972100,884,-300.0,199.0,199.0,6259873,2019-12-18 06:40:19,shipped,170,16,13215,1656835
719954,5363144,2019-12-16 16:25:39,972105,3699,-400.0,223.0,223.0,6455402,2019-12-18 11:26:32,shipped,170,59,35357,1698238
719967,5382115,2019-12-17 08:59:35,968338,4014,-250.0,199.0,199.0,6604544,2019-12-17 12:03:30,shipped,170,30,20540,243971
719987,5415134,2019-12-18 05:42:26,972105,2892,-250.0,199.0,199.0,6817490,2019-12-18 12:17:04,shipped,170,16,8516,1223191


Видим, что скидка не применилась в 37% случаем, так как всего скидка было в 97000 заказов, а в 36000 случаев она не повлияла на цену. Возможно, всё дело в определённых промокодах и механике их работы. Однако это может разочаровать клиента, поэтому стоит обратить внимание на данный вопрос. 

Рассмотрим теперь вопрос о том, как разбиты заказы по городам и ритейлерам. Если где-то будет существенно меньшее значение, то возможно на сайте имеются какие-либо недочёты, из-за которых клиенты не заказывают продукцию данного ритейлера.

In [38]:
stores_id = orders.groupby('store_id')['id'].count() #подготавливаем данные

sub_stores = stores.copy() # создаём копию таблицы для работы с ней

for index in range(162): # в цикле заменяем значения retailer_id на сумму заказов по городам
    if any(stores_id.index.isin(list(sub_stores[sub_stores.index==index]['id'].values))):   
        sub_stores.loc[index,'retailer_id'] = int(stores_id[stores_id.index ==int(sub_stores[sub_stores.index==index]['id'].values)])

print('Разбиение заказов по городам', sub_stores.groupby('city')['retailer_id'].sum())


stores_id = orders.groupby('store_id')['id'].count() # Делаем всё тоже самое, но для суммы заказов по ритейлерам

sub_stores1 = stores.copy()

for index in range(162):
    if any(stores_id.index.isin(list(sub_stores1[sub_stores1.index==index]['id'].values))):   
        sub_stores1.loc[index,'city'] = int(stores_id[stores_id.index ==int(sub_stores1[sub_stores1.index==index]['id'].values)])

print('Разбиение заказов по ритейлерам', sub_stores1.groupby('retailer_id')['city'].sum())

Разбиение заказов по городам city
1     480544
2      39800
5      29424
7      76422
8      28708
10     46812
13     20608
Name: retailer_id, dtype: int64
Разбиение заказов по ритейлерам retailer_id
1     619211
8      53717
15     45405
16      2313
59        14
61         1
Name: city, dtype: int64


Как видим, из результатов анализа, по городам относительно ровное разбиение, однако город с id 1 выделяется, скорее всего это Москва. А вот разбиение по ритейлерам совсем противоположенное. Здесь есть несколько компаний, у которых почти не было заказов. Возможно, они не попали в эту выборку данных, но если дело не в этом, то стоит пересмотреть положение объявления данной компании на сайте или каким-то другим образом напомнить о ней покупателям, возможно даже при взаимном сотрудничестве с самой компанией. 

Теперь посмотрим на то, сколько скидок используют клиенты.

In [39]:
display(orders[orders['promo_total']<0].groupby('user_id')['id'].count().sort_values(ascending=False).head(30))

display(orders[orders['promo_total']<0].groupby('user_id')['id'].count().sort_values(ascending=False).tail(10))

user_id
314205     230
129033     211
339342      46
535160      41
272242      40
308955      39
275376      38
71505       36
287312      36
734540      35
253702      35
105360      33
58839       33
25891       33
222929      30
122506      29
91068       26
314384      25
384875      25
129704      24
339463      24
219341      24
214266      24
249339      23
182307      21
310262      21
1093498     21
32490       21
224310      20
269113      20
Name: id, dtype: int64

user_id
351641     1
351660     1
351689     1
351695     1
351711     1
351713     1
351743     1
351747     1
351752     1
1837043    1
Name: id, dtype: int64

Один клиент применил скидку 230 раз! Возможно, такое количество промокодов на человека слишком щедро со стороны компании - удержать клиента можно и меньшим количество скидок. Другими словами большое количество скидок совсем не оправдано - ведь компания теряет прибыть из-за этих самых скидок. Нужно оптимизировать данный элемент удержания и привлечения. 

# Заключение

На основании приведённых выше исследований можно сделать следующие выводы:

1) Больше половины заказов подвержены замене и/или отмене одного или нескольких товаров, что снижает удовлетворённость клиента. На предоставленных данных не получилось узнать о влиянии таких проблем на удержание клиента. 

2) Проблемы с отсутствием товаров имеют место во всех магазинах, но в различной степени. В некоторых магазинах процент замены товара больше 80, а в некоторых он меньше 50. 

3) Доставка иногда занимает огромное время - конечно, не всегда это именно задержка, иногда клиенты сами хотят, чтобы привезли через неделю или позже, другими словами, оставляют заявку заранее. Но бывают и случаи задержки - это негативно сказывается на удержании клиентов и репутации сервиса. Далеко не все магазины в равной степени имеют такие проблемы, поэтому надо разобраться с этим вопросом. 

4) Некоторые скидки не учитываются при покупках, что ухудшает опыт пользования сервисом. Возможно, здесь имеют место различного рода скидки и купоны, и не все могут уменьшать цены небольших заказов. Однако есть и те скидки, которые работают и на малые заказы. Поэтому проблема с странной механикой скидок всё же существует. И процент таких скидок довольно весом.

5) У некоторых магазинов/ритейлеров совсем мало продаж. Если это соответствует действительности (что можно проверить в полном наборе данных), то имеет смысл пересмотреть маркетинг данных магазинов, как минимум на сайте.

6) Некоторые клиенты используют большое количество скидок. Даже если это "честные" скидки, то всё равно это число можно уменьшить.

На основании приведённых выше исследований можно дать следующие рекомендации компании:

1) Улучшить систему отслеживания товаров на складах так, чтобы при уменьшении ниже определённого уровня на сайте появлялось объявлении о том, что скоро товар закончится, и покупатели не брали его в том случае, если не готовы ждать или не готовы к замене. 

2) Улучшить работу службы доставки, возможно ввести приоритет заказов.

3) Проверить систему скидок, чтобы не было проблем с учитыванием скидок при малых покупках, а также уменьшить количество скидок для пользователей - это не должно сказаться на удержании пользователей. 

4) Проверить причину малой популярности некоторых магазинов, возможно имеет место проблемы на сайте и/или в работе отдела маркетинга.


In [40]:
conn.close()